In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.016620159149169922
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 44.4376064891819
200000 68.33316921137565
300000 66.82646691422076
400000 65.85462397574939
500000 65.90486094916797
600000 68.93298589836327
700000 65.39639103525631
800000 65.32330666675435
900000 65.37664488681004
1000000 64.71127852920894
1100000 20.201306926396803
1200000 66.65250715478047
1300000 65.04548817619334
1400000 64.54021215364934
1500000 64.49878374875045
1600000 67.96766923948346
1700000 66.37875426639383
1800000 64.59392275405672
1900000 65.12367727067446
2000000 65.34011514100352
2100000 51.37550729005453
2200000 35.506019176399356
2300000 63.99808286548207
2400000 63.77943361416201
2500000 62.63670376246403
2600000 63.46159556535019
2700000 66.40447500293654
2800000 62.989995365842496
2900000 62.5860043700234
3000000 62.20851764958713

30100000 71.39170093606883
30200000 67.99400325294249
30300000 68.6529799252997
30400000 55.437795508951
30500000 38.75846637991404
30600000 66.51511526255682
30700000 68.76610931362718
30800000 65.48076303015715
30900000 72.49749188009028
31000000 75.27851333296131
31100000 73.88770175190234
31200000 71.08730774732366
31300000 36.017870504837894
31400000 73.11093353356536
31500000 74.41816010653659
31600000 78.31797491556249
31700000 59.26830199424073
31800000 47.82045489119329
31900000 74.8221467622626
32000000 73.30490128167268
32100000 76.63359449047287
32200000 74.37053924956594
32300000 70.85933468368187
32400000 68.99402694685898
32500000 53.82742034033584
32600000 39.17478649073456
32700000 68.95743505300774
32800000 70.77183829374125
32900000 77.7311907431039
33000000 74.25036986845608
33100000 70.87687580453515
33200000 76.67247539412288
33300000 62.89697908901656
33400000 45.24967837940801
33500000 78.9133015305639
33600000 74.01305536074133
33700000 71.51915856137803
338000

60500000 71.08055659286804
60600000 79.11535361071604
60700000 73.30617354917707
60800000 76.19476464525599
60900000 75.6759450139318
61000000 74.8397935045088
61100000 74.96227899331494
61200000 76.97203133857894
61300000 52.02516082882965
61400000 48.9605204123295
61500000 67.8439140996233
61600000 65.71294389863928
61700000 69.27481758287321
61800000 67.25945651843216
61900000 30.557400993709102
62000000 68.59445990353103
62100000 71.78707288662986
62200000 67.63962710043035
62300000 67.97522487328143
62400000 70.11079121513536
62500000 32.14023629787211
62600000 71.7441412875254
62700000 77.19372421308773
62800000 75.13759481347942
62900000 76.77550494414639
63000000 78.57651841409444
63100000 77.94624568838829
63200000 72.75053212309766
63300000 72.49856498499301
63400000 56.99519108581326
63500000 42.366227973380994
63600000 64.98319776126812
63700000 65.30127061170387
63800000 66.28111286340194
63900000 66.86099478914265
64000000 28.925274220573804
64100000 62.18248276327914
642

90900000 39.10184519847373
91000000 50.19779166941979
91100000 66.10649744188846
91200000 57.4093108830079
91300000 26.358975121666564
91400000 65.97037938537805
91500000 62.31468680527247
91600000 38.2702796582537
91700000 58.05072027973621
91800000 68.25965336981598
91900000 67.2689108969253
92000000 71.49856408048517
92100000 66.07797696870725
92200000 69.73317142001025
92300000 65.76205117442213
92400000 70.97622642869807
92500000 65.7197092033921
92600000 65.16225146912224
92700000 65.95046648164252
92800000 61.321013139538515
92900000 64.30975179100086
93000000 49.83916481730914
93100000 43.150400131978316
93200000 61.90100564241207
93300000 60.26990394593319
93400000 27.17668085581615
93500000 64.84345126893693
93600000 60.67726923470083
93700000 30.102322665409083
93800000 69.67968125195
93900000 63.207787913258436
94000000 63.938507833809055
94100000 66.85302295468414
94200000 69.57007309695436
94300000 71.45764372771947
94400000 72.15380517570937
94500000 73.33251881381386
94

120400000 58.40919210094948
120500000 36.552938599295544
120600000 28.412746274678273
120700000 30.079545065045487
120800000 45.86755517565658
120900000 58.76303999164911
121000000 59.196041373679115
121100000 60.09999735828904
121200000 61.836742057899436
121300000 64.41272525203125
121400000 64.20908729707993
121500000 66.38528883230907
121600000 70.55248703001435
121700000 65.17041121666037
121800000 64.76246800475609
121900000 62.36050624033478
122000000 61.424199555440964
122100000 55.42665831173211
122200000 61.51965507344624
122300000 60.37255162476491
122400000 58.12479457988575
122500000 59.08858010696317
122600000 42.14906063950877
122700000 21.41342810554912
122800000 25.500741297816504
122900000 51.5587984174558
123000000 59.520373104161706
123100000 61.238052076558276
123200000 58.539654771852156
123300000 58.22412670350021
123400000 55.820006284662234
123500000 58.63296590081663
123600000 61.54279504065289
123700000 64.84121203056075
123800000 67.26401026084297
123900000 

...

best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.23544530877501962
799900000 0.26263693485688766
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.2356432649646216
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.5103877506507324
800200000 1.348361646502908
800300000 1.0913632957101822
800400000 1.280094546125657
800500000 1.0370111015693844
800600000 1.068949020622689
800700000 1.0408267322760985
800800000 0.9277419378832816
800900000 1.0546492391821536
801000000 1.0342935748041697
801100000 0.8430917936118424
801200000 1.0191257766854538
801300000 1.2395937873249345
801400000 1.3476163184851886
801500000 0.859730986004279
80

813400000 0.881995250477469
813500000 0.8263549781332619
813600000 0.7837739120956726
813700000 0.9808145626541733
813800000 0.9972057669951857
813900000 0.8250212594266407
814000000 0.8171298866992771
814100000 1.2974087104306056
814200000 0.8991937968339861
814300000 0.8421710037380497
814400000 0.7944642807498229
814500000 0.7702276346498569
814600000 0.8835614841666997
814700000 0.9168898204960042
814800000 0.7861461976458284
814900000 0.8671290208363414
815000000 0.7779209794811863
815100000 0.7932489349602243
815200000 0.8698395963404597
815300000 0.7902677330369791
815400000 0.8950294573189119
815500000 0.8008882857793966
815600000 0.7420269564076517
815700000 0.8666526858078247
815800000 0.7086174147547126
815900000 0.8474326669310007
816000000 0.8371934123347197
816100000 0.7226677238363922
816200000 0.8560805706035038
816300000 0.8144104727448642
816400000 0.674898895922514
816500000 0.955841120899229
816600000 0.6861258785053578
816700000 0.9123672189872324
816800000 0.87159

841000000 0.15146230608311892
841100000 0.2700867084605753
841200000 0.25939129199484057
841300000 0.18226512933365502
841400000 0.2584741198848784
841500000 0.1774711167143868
841600000 0.23389479981883368
841700000 0.25442331514493693
841800000 0.1555620042699867
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.2695960137822628
842000000 0.20680055246209966
842100000 0.1701227697639942
842200000 0.2876276658462187
842300000 0.17025264757666256
842400000 0.24795999226134285
842500000 0.223261244172436
842600000 0.15769739578071568
842700000 0.23251529677490193
842800000 0.18581577318084241
842900000 0.17480425460275673
843000000 0.22439294809885488
843100000 0.15734313994402688
843200000 0.197014728919522
843300000 0.19349528273516234
843400000 0.15112525400908788
843500000 0.225520774873411
843600000 0.1609923210039642
843700000 0.17660261271378994
843800000 0.21442985139692824
843900000 0.14664280070952374
best so far: 0
type: [2, 1, 1, 16, 4] 128
cas

861100000 0.03511393956065774
861200000 0.018768452245087757
861300000 0.05881878638596866
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.045701736612613994
861500000 0.03250575445492731
861600000 0.017255429240803585
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.028968315856385235
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.021943406691064437
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.022436323080778784
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.020265061508846282
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 8, 1] 128
cases of this type: 16384
862100000 0.016322215539881918
best so far: 0
type: [2, 2, 8, 1, 4] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 8, 2, 2] 128
cases of this 

866100000 0.0007494735342495971
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.0002367552138388157
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so f